<a href="https://colab.research.google.com/github/IKAR42/SberDS_Study/blob/main/HW_1_Optuna_Sber_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 1

# Автор: Карайман Игорь

В этой домашней работе Вы сможете попрактиваться в применении фреймворка для подбора гиперпараметров [Optuna](https://optuna.org/) для одной из наиболее популярных вариаций градиентного бустинга [XGBoost](https://xgboost.readthedocs.io/en/stable/python/index.html).

Это домашнее задание состоит из **4х заданий**, **максимальная** сумма за выполнение которых составляет **25 баллов**. Однако, **3 лучших решения** получат дополнительно **+5 бонусных баллов**.

In [1]:
# установка необходимых библиотек

!pip install optuna optuna-dashboard xgboost

In [2]:
# Подключаюсь к диску, чтобы забрать данные для построения модели
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
# Импортирую библиотеки для всей тертадки

import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split

import xgboost as xg
import numpy as np
from sklearn.metrics import mean_squared_error as MSE

import optuna
from sklearn.metrics import accuracy_score

# Задание 1. [5 баллов]

Возьмите выборку на Ваше усмотрение и сделайте ее быстрый анализ, постройте первый вариант модели.

Примеры анализа и построения модели:
* [Предсказание стоимости автомобилей](https://www.kaggle.com/code/niteshyadav3103/car-price-eda-and-modelling). Пример jupyter-ноутбука с детальным анализом данных, разбиением выборки на обучающую и тестовую и построением нескольких классических моделей.
* [Апельсины и грейпфруты](https://www.kaggle.com/code/mahmoudmagdy07/oranges-vs-grapefruit-team-20). Пример jupyter-ноутбука с детальным анализом данных, разбиением выборки на обучающую и тестовую и использованием "Наивного Байеса" и нейросетей.


In [5]:
# Для решения задачи взял самый простой дата сет с Kaggle
# https://www.kaggle.com/datasets/danielkyrka/bmw-pricing-challenge?select=bmw_pricing_challenge.csv

# Мы будем предсказывать стоимость автомобиля BMW на основании ряда характеристик. Данных мало, поэтому качество обучения и предсказания не ожидаются высокими

In [40]:
df_1 = pd.read_csv('drive/My Drive/bmw_pricing_challenge.csv')
df_1.head(3)

,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,price,sold_at
0,BMW,118,140411,100,2012-02-01,diesel,black,convertible,True,True,False,False,True,True,True,False,11300,2018-01-01
1,BMW,M4,13929,317,2016-04-01,petrol,grey,convertible,True,True,False,False,False,True,True,True,69700,2018-02-01
2,BMW,320,183297,120,2012-04-01,diesel,white,convertible,False,False,False,False,True,False,True,False,10200,2018-02-01


In [5]:
df_1.columns

Index(['maker_key', 'model_key', 'mileage', 'engine_power',
       'registration_date', 'fuel', 'paint_color', 'car_type', 'feature_1',
       'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6',
       'feature_7', 'feature_8', 'price', 'sold_at'],
      dtype='object')

In [6]:
# Явных пропусков не наблюдаю
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   maker_key          4843 non-null   object
 1   model_key          4843 non-null   object
 2   mileage            4843 non-null   int64 
 3   engine_power       4843 non-null   int64 
 4   registration_date  4843 non-null   object
 5   fuel               4843 non-null   object
 6   paint_color        4843 non-null   object
 7   car_type           4843 non-null   object
 8   feature_1          4843 non-null   bool  
 9   feature_2          4843 non-null   bool  
 10  feature_3          4843 non-null   bool  
 11  feature_4          4843 non-null   bool  
 12  feature_5          4843 non-null   bool  
 13  feature_6          4843 non-null   bool  
 14  feature_7          4843 non-null   bool  
 15  feature_8          4843 non-null   bool  
 16  price              4843 non-null   int64 


In [7]:
# NaN значений в дата сете не наблюдаю
df_1.isnull().any()

maker_key            False
model_key            False
mileage              False
engine_power         False
registration_date    False
fuel                 False
paint_color          False
car_type             False
feature_1            False
feature_2            False
feature_3            False
feature_4            False
feature_5            False
feature_6            False
feature_7            False
feature_8            False
price                False
sold_at              False
dtype: bool

In [8]:
# Смотрю минимумы и максимумы числовых фич
df_1.describe()

,mileage,engine_power,price
count,4.843000e+03,4843.00000,4843.000000
mean,1.409628e+05,128.98823,15828.081767
std,6.019674e+04,38.99336,9220.285684
min,-6.400000e+01,0.00000,100.000000
25%,1.029135e+05,100.00000,10800.000000
50%,1.410800e+05,120.00000,14200.000000
75%,1.751955e+05,135.00000,18600.000000
max,1.000376e+06,423.00000,178500.000000


In [9]:
df_clean = df_1[['maker_key', 'model_key', 'mileage', 'engine_power', 'registration_date', 'fuel', 'paint_color', 'car_type', 'price']]

In [10]:
# Оставил только нужные столбцы. Все столбцы фичи, кроме столбца price - он будет target'ом для предсказания
df_clean.head(3)

,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,price
0,BMW,118,140411,100,2012-02-01,diesel,black,convertible,11300
1,BMW,M4,13929,317,2016-04-01,petrol,grey,convertible,69700
2,BMW,320,183297,120,2012-04-01,diesel,white,convertible,10200


In [23]:
# Инициализируем нужный метод
le = LabelEncoder()
ohe = OneHotEncoder()

# Формально разделяем выборку по типам переменных
categorical_cols = ['maker_key', 'model_key', 'registration_date', 'fuel', 'paint_color',
            'car_type']
numeric_cols = ['mileage','engine_power', 'price']

In [24]:
# Делаем label encoding для категориальных фич
df_clean[categorical_cols] = df_clean[categorical_cols].apply(lambda col:
le.fit_transform(col))
df_clean.head(2)

<ipython-input-24-f2b802f8e75a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[categorical_cols] = df_clean[categorical_cols].apply(lambda col:


,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,price
0,0,2,140411,100,134,0,1,0,11300
1,0,61,13929,317,184,3,5,0,69700


In [25]:
# Делаем one hot encoding для категориальных фич
array_hot_encoded = ohe.fit_transform(df_clean[categorical_cols])
# Делаем соотвутствуюшее преобразование
data_hot_encoded = pd.DataFrame(array_hot_encoded.todense())

data_numeric_cols = df_clean.drop(df_clean[categorical_cols], axis = 1)

# Собираем полученный результат в один дата сет
df_clean_ohe = pd.concat([data_hot_encoded, data_numeric_cols], axis=1)

In [26]:
# Проверяем полученный результат в виде финального дата сета
df_clean_ohe.head(2)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,mileage,engine_power,price
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140411,100,11300
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13929,317,69700


# 2. Задание 2 [5 баллов]
Разбейте выборку на обучение и контроль. Вам может понадобится [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [27]:
# Выделяю дата сет с фичей и таргет в отдельные переменные
x2 = df_clean_ohe.drop('price',axis=1)
y2 = df_clean_ohe['price']

In [28]:
# Разбиваю дата сет на train и test
x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, test_size=0.2, random_state=42)
x2_train.shape, x2_test.shape, y2_train.shape, y2_test.shape

((3874, 299), (969, 299), (3874,), (969,))

# Задание 3. [10 баллов]

Проведите для данной выборки подбор гиперпараметров для модели `XGBoost` с помощью `Optuna`. Варьируйте **не менее 5** гиперпараметров.


In [107]:
# Объявляю параметры модели
XGB_pricing = xg.XGBRegressor(
                  objective ='reg:linear',
                  n_estimators = 22,
                  learning_rate=0.3,
                  max_depth=10,
                  n_jobs=4,
                  num_parallel_tree=1
                  )

In [108]:
# Обучение и предсказание
XGB_pricing.fit(x2_train, y2_train)
xgb_pred = XGB_pricing.predict(x2_test)

[12:11:11] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [109]:
# Расчет метрик
rmse = np.sqrt(MSE(y2_test, xgb_pred))
print("RMSE : % f" %(rmse))
mse = MSE(y2_test, xgb_pred)
print("MSE : % f" %(mse))

RMSE :  4378.435266
MSE :  19170695.375926


Обучение с Optuna

In [100]:
# Объявляем функцию для Optuna
def objective(trial):

    params = {
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 5, 25),
        'n_jobs': trial.suggest_int('n_jobs', 1, 5),
        'num_parallel_tree': trial.suggest_int('num_parallel_tree', 1, 2)

    }

    optuna_model = xg.XGBRegressor(**params)
    optuna_model.fit(x2_train, y2_train)

    y_pred_op = optuna_model.predict(x2_test)

    mse = MSE(y2_test, y_pred_op)
    return mse

In [ ]:
%%time
# делаем обучение с помощью оптюна / подбираем параметры в направлении минимизации выбранной метрики

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

In [102]:
# узнаем лучшие параметры
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished trials: 50
Best trial:
  Value: 16710426.822891943
  Params: 
    max_depth: 18
    learning_rate: 0.2759559669360914
    n_estimators: 23
    n_jobs: 3
    num_parallel_tree: 2


In [103]:
trial.params

{'max_depth': 18,
 'learning_rate': 0.2759559669360914,
 'n_estimators': 23,
 'n_jobs': 3,
 'num_parallel_tree': 2}

In [104]:
# Определяем полученные параметры для последующег оцикла обучения
best_params_xgboost_optuna_1 = trial.params
best_params_xgboost_optuna_1

{'max_depth': 18,
 'learning_rate': 0.2759559669360914,
 'n_estimators': 23,
 'n_jobs': 3,
 'num_parallel_tree': 2}

In [105]:
# Обучаем модель с наилучшими по версии Optuna параметрами
model_2 = xg.XGBRegressor(**best_params_xgboost_optuna_1)
model_2.fit(x2_train, y2_train)
y_pred_op2 = model_2.predict(x2_test)

In [106]:
# Расчет метрик
rmse = np.sqrt(MSE(y2_test, y_pred_op2))
print("RMSE : % f" %(rmse))
mse = MSE(y2_test, y_pred_op2)
print("MSE : % f" %(mse))

RMSE :  4087.838894
MSE :  16710426.822892


# Задание 4. [5 баллов]

Проанализируйте результаты -- на сколько улучшилось качество модели с учетом Вашего разбиения.

In [110]:
# Расчет метрик без Optuna
rmse = np.sqrt(MSE(y2_test, xgb_pred))
print("RMSE : % f" %(rmse))
mse = MSE(y2_test, xgb_pred)
print("MSE : % f" %(mse))

RMSE :  4378.435266
MSE :  19170695.375926


In [111]:
# Расчет метрик с Optuna
rmse = np.sqrt(MSE(y2_test, y_pred_op2))
print("RMSE : % f" %(rmse))
mse = MSE(y2_test, y_pred_op2)
print("MSE : % f" %(mse))

RMSE :  4087.838894
MSE :  16710426.822892


In [113]:
# Расчет разницы в RMSE
(4087.838894 / 4378.435266) * 100
# Улучшение RMSE на ~ 6,7%

93.3630086013472

In [114]:
# Расчет разницы в MSE
(16710426.822892 / 19170695.375926) * 100
# Улучшение MSE на ~ 12,8%

87.16651376077087

Вывод:
**Результаты полученные с применением Optuna оказались выше, чем рандомный подбор параметров**

*Я осознаю, что полученные метрики на практике гипер аномальны. Здесь я сделал скидку на то, что учебная задача и нужно проследить дельту при построении модели